In [1]:
from sklearn.preprocessing import normalize
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd

In [2]:
#import the data
data_df = pd.read_csv(Path('Resources/crypto_data.csv'))
data_df.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


## Data Preparation

- Reduce data to only cryptocurrencies that are being traded
- Drop 'IsTrading', 'Unnamed: 0' and 'CoinName' columns.
- CoinName was dropped because it does not contribute to the final analysis.
- Drop all null values
- reduce data to only mined cryptocurrencies

In [3]:
#REDUCE DATA TO ONLY TRADED CURRENCIES
traded_crypto_df = data_df.loc[data_df['IsTrading']==True,:]

#verify the drop
print(traded_crypto_df['IsTrading'].unique())
traded_crypto_df.head()

[ True]


,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [4]:
#DROP 'IsTrading', 'Unnamed: 0' and 'CoinName' COLUMNS
#CoinName was dropped because it does not contribute to the final analysis.
traded_crypto_df = traded_crypto_df.drop(['IsTrading', 'CoinName','Unnamed: 0'], axis=1)
print(traded_crypto_df.shape)

traded_crypto_df.head()

(1144, 4)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,Scrypt,PoW/PoS,4.199995e+01,42
1,X11,PoW/PoS,NaN,2300000000
2,Scrypt,PoW/PoS,1.055185e+09,532000000
3,SHA-256,PoW,NaN,611000
4,SHA-256,PoW/PoS,0.000000e+00,0


In [8]:
#DROP ALL NULL VALUES
traded_crypto_df = traded_crypto_df.dropna()
print(traded_crypto_df.shape)

traded_crypto_df.head()

(532, 4)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,Scrypt,PoW/PoS,4.199995e+01,42
2,Scrypt,PoW/PoS,1.055185e+09,532000000
5,X13,PoW/PoS,2.927942e+10,3.14E+11
7,SHA-256,PoW,1.792718e+07,21000000
8,Ethash,PoW,1.076842e+08,0


In [6]:
#REDUCE DATA TO ONLY MINED CRYPTOCURRENCIES
traded_crypto_df = traded_crypto_df.loc[traded_crypto_df['TotalCoinsMined']>0,:]

#verify the changes
over_zero = traded_crypto_df['TotalCoinsMined']>0
print(over_zero.unique())

traded_crypto_df.head()

[ True]


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,Scrypt,PoW/PoS,4.199995e+01,42
2,Scrypt,PoW/PoS,1.055185e+09,532000000
5,X13,PoW/PoS,2.927942e+10,3.14E+11
7,SHA-256,PoW,1.792718e+07,21000000
8,Ethash,PoW,1.076842e+08,0


In [7]:
#CONVERT NON-NUMERICAL VALUES
